# Model Architecture

In [1]:
doc_length = 70
num_encoder_tokens = 8000
num_decoder_tokens = 4500

In [2]:
from keras.layers import Input, Embedding, BatchNormalization, GRU, Dense, merge
from keras.models import Model
from keras import optimizers

/home/richard/deep_learn_p3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
latent_dim = 300

#### Encoder ####
encoder_inputs = Input(shape=(doc_length,), name='Encoder-Input')

x = Embedding(num_encoder_tokens, latent_dim, mask_zero=False, name='Body-Word-Embedding')(encoder_inputs)
x = BatchNormalization(name='Encoder-BatchNorm-1')(x)

_, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)
_, state_h_back = GRU(latent_dim, return_state=True, go_backwards=True, name='Encoder-Last-Backward-GRU')(x)

state_h = merge([state_h, state_h_back], mode='sum')

encoder_model = Model(encoder_inputs, state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

#### Decoder ####
decoder_inputs = Input(shape=(None,), name='Decoder-Input')

x = Embedding(num_decoder_tokens, latent_dim, mask_zero=False, name='Title-Word-Embedding')(decoder_inputs)
x = BatchNormalization(name='Decoder-BatchNorm-1')(x)

decoder_gru = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')

decoder_gru_out, _ = decoder_gru(x, initial_state=seq2seq_encoder_out)

x = BatchNormalization(name='Decoder-BatchNorm-2')(decoder_gru_out)

decoder_dense_out = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')(x)

#### Seq2Seq Model ####
seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_dense_out, name='Seq2Seq-Model')

seq2seq_Model.compile(optimizer=optimizers.Nadam(lr=1e-3), loss='sparse_categorical_crossentropy')

/home/richard/deep_learn_p3/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/home/richard/deep_learn_p3/lib/python3.5/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/richard/deep_learn_p3/lib/python3.5/site-packages/ipykernel_launcher.py:30: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [8]:
seq2seq_Model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Title-Word-Embedding (Embedding (None, None, 300)    1350000     Decoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      (None, 70)           0                                            
__________________________________________________________________________________________________
Decoder-BatchNorm-1 (BatchNorma (None, None, 300)    1200        Title-Word-Embedding[0][0]       
__________________________________________________________________________________________________
Encoder-Mo

In [9]:
from keras.utils import plot_model
plot_model(seq2seq_Model, to_file='model.png')

# Train Model

In [10]:
from Helpers import load_encoder_inputs, load_decoder_inputs

encoder_input_data, _ = load_encoder_inputs('train_body_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_inputs('train_title_vecs.npy')

Shape of encoder input: (1800000, 70)
Shape of decoder input: (1800000, 11)
Shape of decoder target: (1800000, 11)


In [11]:
import numpy as np

batch_size = 1200
epochs = 5
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, -1),
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.12)

Train on 1584000 samples, validate on 216000 samples
Epoch 1/3
1584000/1584000 [==============================] - 517s 326us/step - loss: 2.1862 - val_loss: 1.8347
Epoch 2/3
1584000/1584000 [==============================] - 527s 332us/step - loss: 1.6803 - val_loss: 1.6675
Epoch 3/3
1584000/1584000 [==============================] - 528s 334us/step - loss: 1.5532 - val_loss: 1.6220


In [12]:
seq2seq_Model.save('seq2seq_model.h5')

/home/richard/deep_learn_p3/lib/python3.5/site-packages/keras/engine/topology.py:2379: UserWarning: Layer Decoder-GRU was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder-Model_3/merge_5/add:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '
/home/richard/deep_learn_p3/lib/python3.5/site-packages/keras/engine/topology.py:2379: UserWarning: Layer Decoder-Backward-GRU was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder-Model_3/merge_5/add:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '
